# プロ野球投手の怪我人情報スクレイピング(2025)

In [17]:
# stats_pit.csvの中身を5行確認
import pandas as pd

pd.set_option('display.max_columns', None)   # 全ての列を表示
pd.set_option('display.max_rows', None)      # 全ての行を表示（必要なら）
pd.set_option('display.max_colwidth', None)

df = pd.read_csv('play_info.csv',  encoding="cp932")
df.head()

/var/folders/56/v1rqc2cn6rs7t55b3ql05_0r0000gn/T/ipykernel_10933/3067985060.py:8: DtypeWarning: Columns (125,141) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('play_info.csv',  encoding="cp932")


,game_id,seqno_9,year_id,game_kind_id,game_kind_name,game_date,game_year,game_month,stadium_id,stadium_name,home_team_league_id,home_team_league_name,home_team_id,home_team_name,away_team_league_id,away_team_league_name,away_team_id,away_team_name,play_date,play_time,inning,top_bottom_id,top_bottom_name,pa_of_inning,np_of_pa,pitcher_team_league_id,pitcher_team_league_name,pitcher_team_id,pitcher_team_name,pitcher_id,pitcher_name,pitcher_handedness,is_starter,bf_of_game,np_of_game,batter_team_league_id,batter_team_league_name,batter_team_id,batter_team_name,batter_id,batter_name,batter_handedness,batter_pos,batting_order,is_pinch_hitter,pitch_result,pa_result,run_scored,pickoff_attempt_to,pitch_type_id,pitch_type_name,pitch_type_name_detail,pitch_type_group,pitch_speed,pitch_location_x,pitch_location_y,pitch_target_location_x,pitch_target_location_y,pitch_zone,pitch_zone_side,pitch_zone_height,batted_ball_location_x,batted_ball_location_y,batted_ball_grounded_location_x,batted_ball_grounded_location_y,batted_ball_direction,batted_ball_direction_3,batted_ball_direction_5,batted_ball_distance,batted_ball_pos,batted_ball_sec,batted_ball_type,batted_ball_quality_sec,batted_ball_quality,pre_home_team_score,pre_away_team_score,pre_home_team_score_diff,pre_away_team_score_diff,pre_pitcher_team_score,pre_batter_team_score,pre_pitcher_team_score_diff,pre_batter_team_score_diff,pre_runner_situation,pre_runner_count,pre_ball,pre_strike,pre_out,post_home_team_score,post_away_team_score,post_home_team_score_diff,post_away_team_score_diff,post_pitcher_team_score,post_batter_team_score,post_pitcher_team_score_diff,post_batter_team_score_diff,post_runner_situation,post_runner_count,post_ball,post_strike,post_out,is_risp,count_situation,catcher_id,catcher_name,runner_1b_id,runner_1b_name,runner_2b_id,runner_2b_name,runner_3b_id,runner_3b_name,batter_runner_advanced_to,is_batter_runner_out,runner_1b_advanced_to,is_runner_1b_out,is_runner_1b_sb,is_runner_1b_cs,runner_2b_advanced_to,is_runner_2b_out,is_runner_2b_sb,is_runner_2b_cs,runner_3b_advanced_to,is_runner_3b_out,is_runner_3b_sb,is_runner_3b_cs,fielding_char,fielder_alignment,fielder_1b_id,fielder_1b_name,fielder_2b_id,fielder_2b_name,fielder_3b_id,fielder_3b_name,fielder_ss_id,fielder_ss_name,fielder_lf_id,fielder_lf_name,fielder_cf_id,fielder_cf_name,fielder_rf_id,fielder_rf_name,fielder_dh_id,fielder_dh_name,umpire_id,umpire_name,rec_pitcher_id,rec_pitcher_name,rec_pitcher_handedness,rec_is_starter,rec_batter_id,rec_batter_name,rec_batter_handedness,rec_batter_pos,rec_is_pinch_hitter,is_pitch,is_ball,is_strike,is_called_strike,is_swinging_strike,is_foul,is_swing,is_contact,in_strike_zone,is_opposite_pitch,is_pa,is_ab,is_h,is_1b,is_2b,is_3b,is_hr,tb,rbi,is_bb,is_ibb,is_ubb,is_so,is_hbp,is_sh,is_sf,is_gidp,is_roe,is_fc,is_dp,is_out,is_bbe,is_gb,is_fb,is_ld,is_pu,is_ifh,is_bunt,is_pull,is_straight,is_oppo,is_left,is_right,is_soft,is_med,is_hard,is_productive_out,is_sac_bunt_attempt,is_wp,is_bk,is_pb,is_pk,in_dirt,off_wall,change_situation
0,2021029038,11010101,30022,1,セ・リーグ公式戦,2025-03-28,2025,3,1,東京ドーム,1,セ・リーグ,1,巨人,1,セ・リーグ,2,ヤクルト,2025-03-28,18:19:00,1,1,表,1,1,1,セ・リーグ,1,巨人,1800028,戸郷 翔征,右,1,1,1,1,セ・リーグ,2,ヤクルト,1000100,西川 遥輝,左,9,1,0,ファウル,NaN,0,0,1.0,ストレート,ストレート,直球系,146.0,21.0,177.0,36.0,119.0,2.0,外角,真ん中,-198.0,117.0,NaN,NaN,-59.420773,1.0,1.0,76.661594,NaN,2.0,フライ,0,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,並行,1000176,甲斐 拓也,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,1400101,岡本 和真,1600059,吉川 尚輝,700003,坂本 勇人,2107874,門脇 誠,2000070,若林 楽人,1961425,ヘルナンデス,2118420,キャベッジ,NaN,NaN,9,敷田 直人,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,1,0,0,1,1,1,1,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2021029038,11010201,30022,1,セ・リーグ公式戦,2025-03-28,2025,3,1,東京ドーム,1,セ・リーグ,1,巨人,1,セ・リーグ,2,ヤクルト,2025-03-28,18:19:22.188,1,1,表,1,2,1,セ・リーグ,1,巨人,1800028,戸郷 翔征,右,1,1,2,1,セ・リーグ,2,ヤクルト,1000100,西川 遥輝,左,9,1,0,凡打,一ゴロ,0,0,4.0,スライダー,スライダー,曲がる系,128.0,-14.0,1

## injuriesテーブル
・誰が、いつ、どこを怪我したか
### injuriesテーブルの中身
injury_id INTEGER PRIMARY KEY AUTOINCREMENT
player_name TEXT, 結合キー(CSVの名前と一致)
injury_date TEXT,

body_part TEXT, 部位：'Elbow'(肘), 'Shoulder'(肩), 'Oblique'(脇腹), 'Leg'(脚)など
side TEXT,      患側：'Right'(右), 'Left'(左)
injury_type TEXT, 種類：'Ligament'(靭帯), 'Muscle'(筋肉/肉離れ), 'Impact'(打撲/骨折)

複合ユニーク制約（同じ日の同じ怪我を重複登録しないため）
UNIQUE(player_name, injury_date, original_text)


## daily_statsテーブル
・play_info.csvから作成
・投手の「その日の負荷」
・詳細なデータから「1試合ごとの集計データ」に加工。
### daily_statsテーブルの中身
pitcher_name
pitcher_handedness
game_date

np_of_game(または行数をカウント):総投球数
inning:イニング数
rest_days:(dateから計算)中何日で登板したか

pitch_speed(の平均と最大)
pitch_tipe_name(これから変化球の割合を計算):


In [1]:
import sqlite3
from bs4 import BeautifulSoup
import re

# ---------------------------------------------------------
# 1. 分析用タグ付けロジック (変更なし)
# ---------------------------------------------------------
def categorize_injury(text):
    part = "Other"
    side = "Unknown"
    injury_type = "Other"
    
    clean_text = text.replace("じん帯", "靭帯")

    # 左右
    if "右" in clean_text:
        side = "Right"
    elif "左" in clean_text:
        side = "Left"

    # 部位
    if "肘" in clean_text or "トミー" in clean_text or "TJ" in clean_text:
        part = "Elbow"
    elif "肩" in clean_text:
        part = "Shoulder"
    elif any(w in clean_text for w in ["股関節", "足", "膝", "脚", "ハムストリング", "内転筋", "アキレス"]):
        part = "LowerBody"
    elif any(w in clean_text for w in ["脇腹", "腹斜筋", "腰", "背中"]):
        part = "Core"
    elif "指" in clean_text or "手" in clean_text:
        part = "Hand/Finger"

    # 種類
    if any(w in clean_text for w in ["靭帯", "靱帯", "TJ", "トミー"]):
        injury_type = "Ligament"
    elif any(w in clean_text for w in ["骨折", "骨"]):
        injury_type = "Bone"
    elif any(w in clean_text for w in ["クリーニング", "遊離体", "骨棘"]):
        injury_type = "Cleaning"
    elif any(w in clean_text for w in ["肉離れ", "筋損傷", "炎症", "張り", "コンディション不良", "違和感", "痛"]):
        injury_type = "Muscle/Fatigue"

    return part, side, injury_type

# ---------------------------------------------------------
# 2. データベース初期化 (一旦クリアして作り直します)
# ---------------------------------------------------------
dbname = 'baseball_analysis.db'
conn = sqlite3.connect(dbname)
cur = conn.cursor()

# 既存のテーブルがあれば削除して作り直す（データ重複防止のため）
cur.execute('DROP TABLE IF EXISTS injuries')
cur.execute('''
    CREATE TABLE injuries (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        team TEXT,
        player_name TEXT,
        injury_date TEXT,
        injury_name TEXT,
        body_part TEXT,
        side TEXT,
        injury_type TEXT,
        position TEXT,
        UNIQUE(player_name, injury_date, injury_name)
    )
''')
conn.commit()

# ---------------------------------------------------------
# 3. HTML解析処理
# ---------------------------------------------------------

# ★ここに先ほどのHTML全文を貼ってください
# html_content = """
# <h2 class="wp-block-heading"><span id="toc1">東京ヤクルトスワローズ</span></h2>
# ... (以下略) ...
# """

# ※今回はテスト用に、変数html_contentではなく、
# requestsでURLから取るか、ファイルから読み込む形でもOKです。
# もしrequestsを使うなら以下のブロックのコメントアウトを外してください
import requests
url = "https://www.niwaka-yakyu.info/2025-npb-il-list/"
headers = {"User-Agent": "Mozilla/5.0 ..."} # 適切なUAを設定
response = requests.get(url, headers=headers)
response.encoding = response.apparent_encoding
html_content = response.text

soup = BeautifulSoup(html_content, 'html.parser')
tables = soup.find_all('figure', class_='wp-block-table')

data_list = []
print("解析開始...")

for table in tables:
    # チーム名とポジション判定
    prev_h4 = table.find_previous('h4')
    position_type = prev_h4.get_text(strip=True) if prev_h4 else "不明"
    
    prev_h2 = table.find_previous('h2')
    team_name = prev_h2.get_text(strip=True) if prev_h2 else "チーム不明"

    # 投手のみ抽出
    if "投手" not in position_type:
        continue

    rows = table.find_all('tr')
    current_player = None

    for row in rows:
        # --- 名前行 ---
        badge = row.find('span', class_='badge-blue')
        name_tag = row.find('strong')

        if badge and name_tag:
            raw_name = name_tag.get_text(strip=True)
            # スペース削除
            current_player = raw_name.replace('\u3000', '').replace(' ', '')
            continue

        # --- 怪我情報行 ---
        if current_player:
            text_content = row.get_text()
            
            # 日付抽出（【 】の中身を取り出す）
            date_match = re.search(r'【(.*?)】', text_content)
            injury_tag = row.find('strong')
            
            if date_match and injury_tag:
                raw_date = date_match.group(1) # 例: "24/09/21" または "09/21" または "2024/12月下旬"
                injury_name = injury_tag.get_text(strip=True)
                
                formatted_date = None
                
                # --- 日付判定ロジックの修正箇所 ---
                # パターン1: "24/09/21" のようにスラッシュが2つある場合 -> 2024年など明記されている
                if raw_date.count('/') == 2:
                    parts = raw_date.split('/')
                    # "24" -> "2024"
                    year = "20" + parts[0]
                    month = parts[1].zfill(2)
                    day = parts[2].zfill(2)
                    formatted_date = f"{year}-{month}-{day}"
                
                # パターン2: "09/21" のようにスラッシュが1つの場合 -> 2025年とみなす
                elif raw_date.count('/') == 1:
                    parts = raw_date.split('/')
                    # 形式チェック（数字かどうか）
                    if parts[0].isdigit() and parts[1].isdigit():
                        year = "2025"
                        month = parts[0].zfill(2)
                        day = parts[1].zfill(2)
                        formatted_date = f"{year}-{month}-{day}"
                
                # パターン3: "2024/12月下旬" のような曖昧な表記 -> 無視するかログに出す
                else:
                    print(f"スキップ(曖昧な日付): {current_player} - {raw_date}")
                    continue

                if formatted_date:
                    # 分析用タグ付け
                    part, side, i_type = categorize_injury(injury_name)

                    data_list.append((
                        team_name, current_player, formatted_date, 
                        injury_name, part, side, i_type, "Pitcher"
                    ))
                    print(f"取得: {current_player} ({formatted_date}) - {injury_name}")

# 保存
if data_list:
    cur.executemany('''
        INSERT OR IGNORE INTO injuries 
        (team, player_name, injury_date, injury_name, body_part, side, injury_type, position) 
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    ''', data_list)
    conn.commit()
    print(f"\n合計 {len(data_list)} 件のデータを保存しました。")
else:
    print("データが見つかりませんでした。")

conn.close()

解析開始...
取得: 石山泰稚 (2025-07-12) - 左内腹斜筋肉離れ
取得: 中村優斗 (2025-01-23) - 下半身コンディション不良
取得: 奥川恭伸 (2025-02-18) - 下半身コンディション不良
取得: 星知弥 (2025-02-16) - 下半身コンディション不良
取得: 荘司宏太 (2025-05-08) - コンディション不良
取得: 田口麗斗 (2025-06-27) - 下半身のコンディション不良
取得: ピーター・ランバート (2025-02-16) - 下半身コンディション不良
取得: 高橋奎二 (2025-07-05) - 下半身のコンディション不良
取得: 山下輝 (2025-02-01) - 詳細不明 / キャンプはトレーナー指示
取得: マイケル・バウマン (2025-03-18) - 上半身のコンディション不良
取得: ジョン・デュプランティエ (2025-08-18) - 下肢の張り
取得: 髙橋遥人 (2024-11-09) - 左尺骨短縮術後に対する骨内異物除去術
取得: 漆原大晟 (2025-05-08) - コンディション不良
取得: 畠世周 (2025-04-03) - 右中指のコンディション不良
取得: ニック・ネルソン (2025-03-18) - 下肢の違和感
取得: 島本浩也 (2025-05-07) - 左上肢の張り
取得: 桐敷拓馬 (2025-05-22) - 左上肢の筋疲労
取得: 大竹耕太郎 (2025-02-23) - 下肢の張り
取得: 富田蓮 (2024-11-03) - 右下肢の張り
取得: 石黒佑弥 (2025-06-09) - 脇腹の筋損傷
取得: 湯浅京己 (2024-08-25) - 胸椎黄色靱帯骨化症の手術（国指定難病）
取得: 石井大智 (2025-06-06) - 脳震盪 / 側頭部に打球が当たる
取得: 小川一平 (2022-12-16) - 右肘関節鏡視下手術および尺骨神経前方移行術
取得: 西舘勇陽 (2025-08-02) - 上半身のコンディション不良
取得: 田中千晴 (2024-11-03) - 右肘のクリーニング手術
取得: 泉圭輔 (2025-05-16) - 右肩痛
スキップ(曖昧な日付): 石田充冴 - 4月下旬
取得: 直江大輔 (2